<a href="https://colab.research.google.com/github/skmerick/Bicicles_SB/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANÁLISE EXPLORATORIA DOS DADOS


In [224]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Instalação do pyspark**

In [225]:
!pip install pyspark py4j


**Importação de bibliotecas**

In [226]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.types import *
import pandas as pd


In [227]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

##**Acessando as bases**

In [228]:
pasta_raiz = '/content/drive/MyDrive/datasets/Smarttbot'

**Vendas Ofertas especiais - Sales Special Offer Product**

In [229]:
schema = StructType([
                     StructField("SpecialOfferID", StringType(), True),
                     StructField("ProductID", IntegerType(), False),
                     StructField("rowguid", StringType(), False),
                     StructField("ModifiedDate", TimestampType(), True),
])


df_special_offer = spark.read.csv(f'{pasta_raiz}/Sales_SpecialOfferProduct.csv', sep=';', header=True, schema=schema)

df_special_offer.show()

+--------------+---------+--------------------+-------------------+
|SpecialOfferID|ProductID|             rowguid|       ModifiedDate|
+--------------+---------+--------------------+-------------------+
|             1|      680|BB30B868-D86C-455...|2011-04-01 00:00:00|
|             1|      706|B3C9A4B1-2AE6-4CB...|2011-04-01 00:00:00|
|             1|      707|27B711FE-0B77-4EA...|2011-04-01 00:00:00|
|             1|      708|46CBB78B-246E-4D6...|2011-04-01 00:00:00|
|             1|      709|CF102AA0-055F-4D2...|2011-04-01 00:00:00|
|             1|      710|63718DA1-464B-432...|2011-04-01 00:00:00|
|             1|      711|457EB971-D1C9-48C...|2011-04-01 00:00:00|
|             1|      712|5B948448-BAE5-4F2...|2011-04-01 00:00:00|
|             1|      713|07768F40-6E46-430...|2011-04-01 00:00:00|
|             1|      714|85004BCE-C74A-4D4...|2011-04-01 00:00:00|
|             1|      715|BE469616-B279-452...|2011-04-01 00:00:00|
|             1|      716|96D141FD-1D8E-431...|2

In [230]:
df_special_offer.printSchema()

root
 |-- SpecialOfferID: string (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [231]:
#Quantidade de linhas e colunas

def sparkShape(dataframe):
    return(dataframe.count(), len(dataframe.columns))

sparkShape(df_special_offer)

(538, 4)

In [232]:
#Detalhes estatisticos gerais
df_special_offer.describe().show()

+-------+------------------+-----------------+--------------------+
|summary|    SpecialOfferID|        ProductID|             rowguid|
+-------+------------------+-----------------+--------------------+
|  count|               538|              538|                 538|
|   mean|2.7118959107806693|849.4702602230483|                null|
| stddev| 3.478142408113732|86.58989606682316|                null|
|    min|                 1|              680|0020931C-087C-42F...|
|    max|                 9|              999|FFE24AE4-9E46-433...|
+-------+------------------+-----------------+--------------------+



In [233]:
#Quantidade de produtos unicos
df_special_offer.select(countDistinct("productid")).show()

+-------------------------+
|count(DISTINCT productid)|
+-------------------------+
|                      295|
+-------------------------+



In [234]:
df_special_offer.select('specialofferid').groupby('specialofferid').count().show()

+--------------+-----+
|specialofferid|count|
+--------------+-----+
|             7|    8|
|            15|    7|
|            11|    3|
|             3|   55|
|             8|    3|
|            16|    7|
|             5|    2|
|             9|    1|
|             1|  295|
|            10|    3|
|             4|   17|
|            12|   12|
|            13|   10|
|            14|    4|
|             2|  111|
+--------------+-----+



In [235]:
df_special_offer.select('modifieddate').groupby('modifieddate').count().show()

+-------------------+-----+
|       modifieddate|count|
+-------------------+-----+
|2012-04-30 00:00:00|    4|
|2013-04-30 00:00:00|   29|
|2011-04-01 00:00:00|  295|
|2014-03-01 00:00:00|    7|
|2013-06-14 00:00:00|    7|
|2012-03-14 00:00:00|    8|
|2011-05-01 00:00:00|  185|
|2013-04-14 00:00:00|    3|
+-------------------+-----+



**Produtos Produção - Production Product**

In [236]:
schema = StructType([
                     StructField("ProductId", StringType(), False),
                     StructField("Name", StringType(), False),
                     StructField("ProductNumber", StringType(), False),
                     StructField("MakeFlag", IntegerType(), True),
                     StructField("FinishedGoodsFlag", IntegerType(), True),
                     StructField("Color", StringType(), True),
                     StructField("SafetyStockLevel", IntegerType(), True),
                     StructField("ReorderPoint", IntegerType(), True),
                     StructField("StandardCost", StringType(), True),
                     StructField("ListPrice", StringType(), True),
                     StructField("size", StringType(), True),
                     StructField("SizeUnitMeasureCode", StringType(), True),
                     StructField("WeightUnitMeasureCode", StringType(), True),
                     StructField("Weight", StringType(), True),
                     StructField("DaysToManufacture", IntegerType(), True),
                     StructField("ProductLine", StringType(), True),
                     StructField("Class", StringType(), True),
                     StructField("Style", StringType(), True),
                     StructField("ProductSubcategoryID", StringType(), True),
                     StructField("ProductModelID", StringType(), True),
                     StructField("SellStartDate", TimestampType(), True),
                     StructField("SellEndDate", TimestampType(), True),
                     StructField("DiscontinuedDate", TimestampType(), False),
                     StructField("rowguid", StringType(), True),
                     StructField("ModifiedDate", TimestampType(), True),
])
df_product = spark.read.csv(f'{pasta_raiz}/Production_Product.csv', sep=';', header=True, schema=schema)

df_product = df_product.withColumn('StandardCost', regexp_replace('StandardCost', ',', '.').cast('float'))
df_product = df_product.withColumn('ListPrice', regexp_replace('ListPrice', ',', '.').cast('float'))

df_product.show()

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductId|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

In [237]:
df_product.printSchema()

root
 |-- ProductId: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- MakeFlag: integer (nullable = true)
 |-- FinishedGoodsFlag: integer (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: integer (nullable = true)
 |-- ReorderPoint: integer (nullable = true)
 |-- StandardCost: float (nullable = true)
 |-- ListPrice: float (nullable = true)
 |-- size: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- DaysToManufacture: integer (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- ProductSubcategoryID: string (nullable = true)
 |-- ProductModelID: string (nullable = true)
 |-- SellStartDate: timestamp (nullable = true)
 |-- SellEndDate: timestamp (nullable = true)
 |-- DiscontinuedDate: timestam

In [238]:
#Quantidade de linhas e colunas

sparkShape(df_product)

(504, 25)

In [239]:
#Detalhes estatisticos gerais
df_product.describe().show()

+-------+------------------+-----------------+-------------+------------------+-------------------+------+-----------------+------------------+-----------------+-----------------+------------------+-------------------+---------------------+------------------+------------------+-----------+-----+-----+--------------------+-----------------+--------------------+
|summary|         ProductId|             Name|ProductNumber|          MakeFlag|  FinishedGoodsFlag| Color| SafetyStockLevel|      ReorderPoint|     StandardCost|        ListPrice|              size|SizeUnitMeasureCode|WeightUnitMeasureCode|            Weight| DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|   ProductModelID|             rowguid|
+-------+------------------+-----------------+-------------+------------------+-------------------+------+-----------------+------------------+-----------------+-----------------+------------------+-------------------+---------------------+------------------+---------------

In [240]:
#Quantidade de produtos unicos
df_product.select(countDistinct("productid")).show()

+-------------------------+
|count(DISTINCT productid)|
+-------------------------+
|                      504|
+-------------------------+



**Pedidos - Sales Order Header**

In [241]:
schema = StructType([
                     StructField("SalesOrderID", StringType(), True),
                     StructField("RevisionNumber", IntegerType(), True),
                     StructField("OrderDate", TimestampType(), True),
                     StructField("DueDate", TimestampType(), True),
                     StructField("ShipDate", TimestampType(), True),
                     StructField("Status", IntegerType(), True),
                     StructField("OnlineOrderFlag", IntegerType(), True),
                     StructField("SalesOrderNumber", StringType(), True),
                     StructField("PurchaseOrderNumber", StringType(), True),
                     StructField("AccountNumber", StringType(), True),
                     StructField("CustomerID", StringType(), True),
                     StructField("SalesPersonID", StringType(), True),
                     StructField("TerritoryID", StringType(), True),
                     StructField("BillToAdressID", StringType(), True),
                     StructField("ShipToAdressId", StringType(), True),
                     StructField("ShipMethodID", StringType(), True),
                     StructField("CreditcardID", StringType(), True),
                     StructField("CredicardApprovalCode", StringType(), True),
                     StructField("CurrencyRateId", StringType(), True),
                     StructField("SubTotal", StringType(), True),
                     StructField("TaxAmt", StringType(), True),
                     StructField("Freight", StringType(), True),
                     StructField("TotalDue", StringType(), True),
                     StructField("Comment", StringType(), True),
                     StructField("rowguid", StringType(), True),
                     StructField("ModifiedDate", TimestampType(), True),

])

df_order_header = spark.read.csv(f'{pasta_raiz}/Sales_SalesOrderHeader.csv', sep=';', header=True, schema=schema)

df_order_header = df_order_header.withColumn('SubTotal', regexp_replace('TotalDue', ',', '.').cast('float'))
df_order_header = df_order_header.withColumn('TaxAmt', regexp_replace('TotalDue', ',', '.').cast('float'))
df_order_header = df_order_header.withColumn('Freight', regexp_replace('TotalDue', ',', '.').cast('float'))
df_order_header = df_order_header.withColumn('TotalDue', regexp_replace('TotalDue', ',', '.').cast('float'))

df_order_header.show()

+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+--------------+--------------+------------+------------+---------------------+--------------+---------+---------+---------+---------+-------+--------------------+-------------------+
|SalesOrderID|RevisionNumber|          OrderDate|            DueDate|           ShipDate|Status|OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|CustomerID|SalesPersonID|TerritoryID|BillToAdressID|ShipToAdressId|ShipMethodID|CreditcardID|CredicardApprovalCode|CurrencyRateId| SubTotal|   TaxAmt|  Freight| TotalDue|Comment|             rowguid|       ModifiedDate|
+------------+--------------+-------------------+-------------------+-------------------+------+---------------+----------------+-------------------+--------------+----------+-------------+-----------+--------------+--------------+-

In [242]:
df_order_header.printSchema()

root
 |-- SalesOrderID: string (nullable = true)
 |-- RevisionNumber: integer (nullable = true)
 |-- OrderDate: timestamp (nullable = true)
 |-- DueDate: timestamp (nullable = true)
 |-- ShipDate: timestamp (nullable = true)
 |-- Status: integer (nullable = true)
 |-- OnlineOrderFlag: integer (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- PurchaseOrderNumber: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- SalesPersonID: string (nullable = true)
 |-- TerritoryID: string (nullable = true)
 |-- BillToAdressID: string (nullable = true)
 |-- ShipToAdressId: string (nullable = true)
 |-- ShipMethodID: string (nullable = true)
 |-- CreditcardID: string (nullable = true)
 |-- CredicardApprovalCode: string (nullable = true)
 |-- CurrencyRateId: string (nullable = true)
 |-- SubTotal: float (nullable = true)
 |-- TaxAmt: float (nullable = true)
 |-- Freight: float (nullable = true)
 |-- TotalDue: float 

In [243]:
#Quantidade de linhas e colunas

sparkShape(df_order_header)

(31465, 26)

In [244]:
#Detalhes estatisticos gerais
df_order_header.describe().show()

+-------+-----------------+-------------------+------+-------------------+----------------+-------------------+--------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+---------------------+-----------------+------------------+------------------+------------------+------------------+-------+--------------------+
|summary|     SalesOrderID|     RevisionNumber|Status|    OnlineOrderFlag|SalesOrderNumber|PurchaseOrderNumber| AccountNumber|       CustomerID|    SalesPersonID|      TerritoryID|    BillToAdressID|    ShipToAdressId|      ShipMethodID|     CreditcardID|CredicardApprovalCode|   CurrencyRateId|          SubTotal|            TaxAmt|           Freight|          TotalDue|Comment|             rowguid|
+-------+-----------------+-------------------+------+-------------------+----------------+-------------------+--------------+-----------------+-----------------+-----------------+------------------

In [245]:
#CustomerId com maior numero de pedidos.
df_order_header.select("CustomerId").groupby("customerid").count().sort('count', ascending=False).show()

+----------+-----+
|customerid|count|
+----------+-----+
|     11091|   28|
|     11176|   28|
|     11277|   27|
|     11711|   27|
|     11331|   27|
|     11300|   27|
|     11185|   27|
|     11287|   27|
|     11276|   27|
|     11330|   27|
|     11223|   27|
|     11200|   27|
|     11262|   27|
|     11566|   25|
|     11078|   17|
|     11019|   17|
|     11203|   17|
|     11142|   17|
|     11212|   17|
|     11215|   17|
+----------+-----+
only showing top 20 rows



In [246]:
#Maiores valores de pedidos
df_order_header.select("SalesOrderId", "TotalDue").orderBy("TotalDue", ascending=False).show()


+------------+----------+
|SalesOrderId|  TotalDue|
+------------+----------+
|       51131| 187487.83|
|       55282| 182018.62|
|       46616| 170512.67|
|       46981| 166537.08|
|       47395| 165028.75|
|       47369| 158056.55|
|       47355| 145741.86|
|       51822| 145454.36|
|       44518| 142312.22|
|       51858| 140042.12|
|       57150| 137721.31|
|       43875| 137343.28|
|       46607| 135606.67|
|       46660| 132727.84|
|       67305| 130907.05|
|       43884|130416.484|
|       53573| 130249.26|
|       47455| 126992.22|
|       51830|126852.164|
|       47441| 126830.96|
+------------+----------+
only showing top 20 rows



**Clientes - Sales.Customer**

In [247]:
schema = StructType([
                     StructField("CustomerID", StringType(), True),
                     StructField("PersonID", StringType(), True),
                     StructField("StoreID", StringType(), True),
                     StructField("TerritoryID", StringType(), True),
                     StructField("AccountNumber", StringType(), True),
                     StructField("rowguid", StringType(), True),
                     StructField("ModifiedDate", TimestampType(), True),

])

df_customers = spark.read.csv(f'{pasta_raiz}/Sales_Customer.csv', sep=';', header=True, schema=schema)

df_customers.show()

+----------+--------+-------+-----------+-------------+--------------------+--------------------+
|CustomerID|PersonID|StoreID|TerritoryID|AccountNumber|             rowguid|        ModifiedDate|
+----------+--------+-------+-----------+-------------+--------------------+--------------------+
|         1|    NULL|    934|          1|   AW00000001|3F5AE95E-B87D-4AE...|2014-09-12 11:15:...|
|         2|    NULL|   1028|          1|   AW00000002|E552F657-A9AF-4A7...|2014-09-12 11:15:...|
|         3|    NULL|    642|          4|   AW00000003|130774B1-DB21-4EF...|2014-09-12 11:15:...|
|         4|    NULL|    932|          4|   AW00000004|FF862851-1DAA-404...|2014-09-12 11:15:...|
|         5|    NULL|   1026|          4|   AW00000005|83905BDC-6F5E-4F7...|2014-09-12 11:15:...|
|         6|    NULL|    644|          4|   AW00000006|1A92DF88-BFA2-467...|2014-09-12 11:15:...|
|         7|    NULL|    930|          1|   AW00000007|03E9273E-B193-448...|2014-09-12 11:15:...|
|         8|    NULL

In [248]:
df_customers.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- PersonID: string (nullable = true)
 |-- StoreID: string (nullable = true)
 |-- TerritoryID: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [249]:
#Quantidade de linhas e colunas

sparkShape(df_customers)

(19820, 7)

In [250]:
#Detalhes estatisticos gerais
df_customers.describe().show()

+-------+------------------+-----------------+------------------+------------------+-------------+--------------------+
|summary|        CustomerID|         PersonID|           StoreID|       TerritoryID|AccountNumber|             rowguid|
+-------+------------------+-----------------+------------------+------------------+-------------+--------------------+
|  count|             19820|            19820|             19820|             19820|        19820|               19820|
|   mean|  19844.2770938446|11184.19022961452|1037.6549401197606|  5.82497477295661|         null|                null|
| stddev|6581.7859142707575| 5578.70597685964| 475.9147548439723|3.0426757383909195|         null|                null|
|    min|                 1|            10000|              1000|                 1|   AW00000001|0006E071-D04E-426...|
|    max|                99|             NULL|              NULL|                 9|   AW00030118|FFFFF252-5BFC-482...|
+-------+------------------+------------

In [251]:
schema = StructType([
                     StructField("BusinessEntityID", StringType(), True),
                     StructField("PersonType", StringType(), True),
                     StructField("NameStyle", StringType(), True),
                     StructField("Title", StringType(), True),
                     StructField("FirstName", StringType(), True),
                     StructField("MiddleName", StringType(), True),
                     StructField("LastName", StringType(), True),
                     StructField("Suffix", StringType(), True),
                     StructField("EmailPromotion", StringType(), True),
                     StructField("AdditionalContactInfo", StringType(), True),
                     StructField("Demographics", StringType(), True),
                     StructField("rowguid", StringType(), True),
                     StructField("ModifiedDate", TimestampType(), True),

])

df_person = spark.read.csv(f'{pasta_raiz}/Person_Person.csv', sep=';', header=True, schema=schema)

df_person.show()

+----------------+----------+---------+-----+---------+----------+----------+------+--------------+---------------------+--------------------+--------------------+-------------------+
|BusinessEntityID|PersonType|NameStyle|Title|FirstName|MiddleName|  LastName|Suffix|EmailPromotion|AdditionalContactInfo|        Demographics|             rowguid|       ModifiedDate|
+----------------+----------+---------+-----+---------+----------+----------+------+--------------+---------------------+--------------------+--------------------+-------------------+
|               1|        EM|        0| NULL|      Ken|         J|   Sánchez|  NULL|             0|                 NULL|"<IndividualSurve...|92C4279F-1207-48A...|2009-01-07 00:00:00|
|               2|        EM|        0| NULL|    Terri|       Lee|     Duffy|  NULL|             1|                 NULL|"<IndividualSurve...|D8763459-8AA8-47C...|2008-01-24 00:00:00|
|               3|        EM|        0| NULL|  Roberto|      NULL|Tamburello|  N

In [252]:
df_person.printSchema()

root
 |-- BusinessEntityID: string (nullable = true)
 |-- PersonType: string (nullable = true)
 |-- NameStyle: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Suffix: string (nullable = true)
 |-- EmailPromotion: string (nullable = true)
 |-- AdditionalContactInfo: string (nullable = true)
 |-- Demographics: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [253]:
#Quantidade de linhas e colunas

sparkShape(df_person)

(19972, 13)

In [254]:
#Detalhes estatisticos gerais
df_person.describe().show()

+-------+------------------+----------+---------+-----+---------+----------+--------+------+------------------+---------------------+--------------------+--------------------+
|summary|  BusinessEntityID|PersonType|NameStyle|Title|FirstName|MiddleName|LastName|Suffix|    EmailPromotion|AdditionalContactInfo|        Demographics|             rowguid|
+-------+------------------+----------+---------+-----+---------+----------+--------+------+------------------+---------------------+--------------------+--------------------+
|  count|             19972|     19972|    19972|19972|    19972|     19972|   19972| 19972|             19972|                19972|               19972|               19972|
|   mean|10763.079411175646|      null|      0.0| null|     null|      null|    null|  null|0.6300821149609453|                 null|                null|                null|
| stddev|5814.1332719480615|      null|      0.0| null|     null|      null|    null|  null|0.7814331436634205|         

**Detalhes Pedidos = SalesOrderDetails**

In [255]:
schema = StructType([
                     StructField("SalesOrderID", StringType(), True),
                     StructField("SalesOrderDetailID", StringType(), True),
                     StructField("CarrierTrackingNumber", StringType(), True),
                     StructField("OrderQty", StringType(), True),
                     StructField("ProductID", StringType(), True),
                     StructField("SpecialOfferID", StringType(), True),
                     StructField("UnitPrice", StringType(), True),
                     StructField("UnitPriceDiscount", StringType(), True),
                     StructField("LineTotal", FloatType(), True),
                     StructField("rowguid", StringType(), True),
                     StructField("ModifiedDate", TimestampType(), True),

])

df_details = spark.read.csv(f'{pasta_raiz}/Sales_SalesOrderDetail.csv', sep=';', header=True, schema=schema)

df_details = df_details.withColumn('UnitPrice', regexp_replace('UnitPrice', ',', '.').cast('float'))
df_details = df_details.withColumn('UnitPriceDiscount', regexp_replace('UnitPriceDiscount', ',', '.').cast('float'))

df_details.show()

+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|SalesOrderID|SalesOrderDetailID|CarrierTrackingNumber|OrderQty|ProductID|SpecialOfferID|UnitPrice|UnitPriceDiscount|LineTotal|             rowguid|       ModifiedDate|
+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|       43659|                 1|         4911-403C-98|       1|      776|             1| 2024.994|              0.0| 2024.994|B207C96D-D9E6-402...|2011-05-31 00:00:00|
|       43659|                 2|         4911-403C-98|       3|      777|             1| 2024.994|              0.0| 6074.982|7ABB600D-1E77-41B...|2011-05-31 00:00:00|
|       43659|                 3|         4911-403C-98|       1|      778|             1| 2024.994|              0.0| 2024.994|475CF8C6-49F6-486...|2011-05

In [256]:
df_details.printSchema()

root
 |-- SalesOrderID: string (nullable = true)
 |-- SalesOrderDetailID: string (nullable = true)
 |-- CarrierTrackingNumber: string (nullable = true)
 |-- OrderQty: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- SpecialOfferID: string (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- UnitPriceDiscount: float (nullable = true)
 |-- LineTotal: float (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [257]:
#Quantidade de linhas e colunas

sparkShape(df_details)

(121317, 11)

In [258]:
#Detalhes estatisticos gerais
df_details.describe().show()

+-------+------------------+------------------+---------------------+------------------+-----------------+------------------+------------------+-------------------+------------------+--------------------+
|summary|      SalesOrderID|SalesOrderDetailID|CarrierTrackingNumber|          OrderQty|        ProductID|    SpecialOfferID|         UnitPrice|  UnitPriceDiscount|         LineTotal|             rowguid|
+-------+------------------+------------------+---------------------+------------------+-----------------+------------------+------------------+-------------------+------------------+--------------------+
|  count|            121317|            121317|               121317|            121317|           121317|            121317|            121317|             121317|            121317|              121317|
|   mean|57827.363782487206|           60659.0|                 null|2.2660797744751355|841.6808361565156|1.1625411113034447|465.09349706117285|0.00282606728862525| 905.44920693995

## **Análise de dados**

In [259]:
# DataFrames

df_special_offer
df_product
df_order_header
df_customers
df_person
df_details

# SQL

df_special_offer.createOrReplaceTempView('SPECIAL')
df_product.createOrReplaceTempView('PRODUCT')
df_order_header.createOrReplaceTempView('ORDER')
df_customers.createOrReplaceTempView('CUSTOMERS')
df_person.createOrReplaceTempView('PERSON')
df_details.createOrReplaceTempView('DETAILS')


Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.

In [260]:
df_details.show()

+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|SalesOrderID|SalesOrderDetailID|CarrierTrackingNumber|OrderQty|ProductID|SpecialOfferID|UnitPrice|UnitPriceDiscount|LineTotal|             rowguid|       ModifiedDate|
+------------+------------------+---------------------+--------+---------+--------------+---------+-----------------+---------+--------------------+-------------------+
|       43659|                 1|         4911-403C-98|       1|      776|             1| 2024.994|              0.0| 2024.994|B207C96D-D9E6-402...|2011-05-31 00:00:00|
|       43659|                 2|         4911-403C-98|       3|      777|             1| 2024.994|              0.0| 6074.982|7ABB600D-1E77-41B...|2011-05-31 00:00:00|
|       43659|                 3|         4911-403C-98|       1|      778|             1| 2024.994|              0.0| 2024.994|475CF8C6-49F6-486...|2011-05

In [261]:
spark.sql('''SELECT COUNT(*) AS TOTAL 
FROM (SELECT SalesOrderID, count(*) AS D1 
        FROM DETAILS 
        GROUP BY SalesOrderID) 
WHERE D1 >= 3''').show()

+-----+
|TOTAL|
+-----+
|12757|
+-----+



 Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct
e Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de
OrderQty), agrupados pelo número de dias para manufatura (DaysToManufacture).

In [262]:
spark.sql('''SELECT PRODUCT.NAME, SUM(DETAILS.ORDERQTY) AS SOMA, PRODUCT.DAYSTOMANUFACTURE 
FROM PRODUCT 
LEFT JOIN SPECIAL ON PRODUCT.PRODUCTID = SPECIAL.PRODUCTID 
LEFT JOIN DETAILS ON SPECIAL.SPECIALOFFERID = DETAILS.SPECIALOFFERID
GROUP BY PRODUCT.NAME, PRODUCT.DAYSTOMANUFACTURE 
ORDER BY SOMA DESC
LIMIT 3''').show()

+--------------------+--------+-----------------+
|                NAME|    SOMA|DAYSTOMANUFACTURE|
+--------------------+--------+-----------------+
|Full-Finger Glove...|270244.0|                0|
|Women's Mountain ...|270244.0|                0|
|     Classic Vest, S|270159.0|                0|
+--------------------+--------+-----------------+



Escreva uma query ligando as tabelas Person.Person, Sales.Customer e
Sales.SalesOrderHeader de forma a obter uma lista de nomes de clientes e uma contagem
de pedidos efetuados.

In [263]:
spark.sql('''SELECT PERSON.FIRSTNAME, COUNT(*) AS PEDIDOS
FROM PERSON 
INNER JOIN ORDER ON PERSON.BUSINESSENTITYID = ORDER.CUSTOMERID 
GROUP BY PERSON.FIRSTNAME
''').show()

+---------+-------+
|FIRSTNAME|PEDIDOS|
+---------+-------+
| Samantha|     77|
|   Damien|     34|
|    Ruben|    163|
|   Wilson|      2|
|    Grace|    177|
|    Lucas|     75|
|   Gerald|     33|
|    Edwin|     92|
|    Robyn|     39|
|   Willie|     63|
|     Mary|      9|
| Isabella|    118|
|    James|     71|
|    Jaime|      6|
|   Rakesh|      2|
|    Wyatt|     42|
|   Trevor|     35|
|  Antonio|     43|
|  Natalie|    114|
|      Roy|     91|
+---------+-------+
only showing top 20 rows



Escreva uma query usando as tabelas Sales.SalesOrderHeader, Sales.SalesOrderDetail e
Production.Product, de forma a obter a soma total de produtos (OrderQty) por ProductID e
OrderDate.

In [264]:
spark.sql('''
SELECT PRODUCT.PRODUCTID, ORDER.ORDERDATE, SUM(DETAILS.ORDERQTY) AS SOMA
FROM PRODUCT
JOIN SPECIAL
ON PRODUCT.PRODUCTID = SPECIAL.PRODUCTID
JOIN DETAILS
ON SPECIAL.SPECIALOFFERID = DETAILS.SPECIALOFFERID
JOIN ORDER
ON DETAILS.SALESORDERID = ORDER.SALESORDERID
GROUP BY PRODUCT.PRODUCTID, ORDER.ORDERDATE
''').show()

+---------+-------------------+----+
|PRODUCTID|          ORDERDATE|SOMA|
+---------+-------------------+----+
|      715|2014-06-09 00:00:00|69.0|
|      717|2014-06-11 00:00:00|61.0|
|      730|2014-06-28 00:00:00|68.0|
|      736|2014-06-23 00:00:00|82.0|
|      739|2014-06-14 00:00:00|78.0|
|      741|2014-06-04 00:00:00|62.0|
|      744|2014-06-19 00:00:00|79.0|
|      749|2014-06-10 00:00:00|63.0|
|      754|2014-06-25 00:00:00|75.0|
|      760|2014-06-27 00:00:00|82.0|
|      761|2014-06-03 00:00:00|53.0|
|      764|2014-06-25 00:00:00|75.0|
|      774|2014-06-29 00:00:00|61.0|
|      786|2014-06-14 00:00:00|78.0|
|      789|2014-06-22 00:00:00|69.0|
|      789|2014-06-09 00:00:00|69.0|
|      789|2014-05-31 00:00:00|79.0|
|      792|2014-06-23 00:00:00|82.0|
|      793|2014-06-11 00:00:00|61.0|
|      794|2014-06-27 00:00:00|82.0|
+---------+-------------------+----+
only showing top 20 rows



Escreva uma query mostrando os campos SalesOrderID, OrderDate e TotalDue da tabela
Sales.SalesOrderHeader. Obtenha apenas as linhas onde a ordem tenha sido feita durante
o mês de setembro/2011 e o total devido esteja acima de 1.000. Ordene pelo total devido
decrescente.

In [265]:
spark.sql('''
SELECT SALESORDERID, ORDERDATE, TOTALDUE 
FROM ORDER
WHERE ORDERDATE >='2011-09-01' AND ORDERDATE <'2011-10-01'  AND TOTALDUE > 1000
ORDER BY TOTALDUE DESC
''').show()

+------------+-------------------+---------+
|SALESORDERID|          ORDERDATE| TOTALDUE|
+------------+-------------------+---------+
|       44348|2011-09-07 00:00:00|3953.9883|
|       44372|2011-09-09 00:00:00|3953.9883|
|       44349|2011-09-07 00:00:00|3953.9883|
|       44350|2011-09-07 00:00:00|3953.9883|
|       44371|2011-09-09 00:00:00|3953.9883|
|       44351|2011-09-07 00:00:00|3953.9883|
|       44328|2011-09-02 00:00:00|3953.9883|
|       44352|2011-09-07 00:00:00|3953.9883|
|       44330|2011-09-02 00:00:00|3953.9883|
|       44332|2011-09-03 00:00:00|3953.9883|
|       44370|2011-09-09 00:00:00|3953.9883|
|       44357|2011-09-07 00:00:00|3953.9883|
|       44338|2011-09-04 00:00:00|3953.9883|
|       44358|2011-09-07 00:00:00|3953.9883|
|       44340|2011-09-04 00:00:00|3953.9883|
|       44359|2011-09-08 00:00:00|3953.9883|
|       44344|2011-09-06 00:00:00|3953.9883|
|       44360|2011-09-08 00:00:00|3953.9883|
|       44347|2011-09-06 00:00:00|3953.9883|
|       44